# Diversity di uno shot

In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample


In [2]:
tags_found = []

with open("/home/simo/UnInfo/Tesi Magistrale/Dataset/features/shots_tags_frequences.txt", "r") as ins:
    for line in ins:

        tag = line[0:line.rfind('-')-1]
        count = line[line.rfind('-')+2:len(line)-1]
        elem = {'tag': tag.decode('utf-8'), 'count': int(count)}
        tags_found.append(elem)

In [9]:
from decimal import *
getcontext().prec = 1000



# ritorna lo shot dato il suoi id
def get_shot(shot_id):
    shot = db.shots.find(
            {'shots': {
            '$all':[{
                    '$elemMatch': {'id': shot_id}
                    }]
                }}
        )

    for s in shot[0]['shots']:
        if s['id'] == shot_id:
            return s

        
def Utf8DecodeArray( oldArray ):  
    newArray = []  
    for element in oldArray:  
        if isinstance(element, unicode):  
            newArray.append(element.decode("utf-8"))  
        else:  
            newArray.append(element)  
    return str(newArray)  
        
        
# clacola la prob che il tag 'tag' sia contenuto in uno shot 
# 578143 = totale degli shots CONSIDERATI
def tag_prob(tag):
    try:
        tag = tag.replace(u'\xe1', u'a')
    except:
        print 'Shot_diversity -> UnicodeDecodeError'
    tag = tag.replace(u'\xa0', u'')
    tag = tag.replace(u'，', u',')
    for t in tags_found:   
        if t['tag'] == tag or (t['tag'] in tag):            
            if float(Decimal(t['count'])/Decimal(578143)) <= float(0.001):
                return float(0)
            else:
                return round(float(Decimal(t['count'])/Decimal(578143)),3)
    print 'shot_diversity - string encoding error:'+tag
    return 0

# calcola l'informazione di un tag
def tag_info(tag):
    p_tag = tag_prob(tag)
    if p_tag == 0:
        return 0
    else:
        return (float(p_tag) * np.log2(float(p_tag)))*-1

# Calcolo della Diversity
def diversity(shot_tags):
    d = 0
    #tags = get_shot(shot_id)['tags']
    #tags = db.shots_single.find({'id':shot_id})[0]['tags']
    for t in shot_tags:
        d = d + (float(tag_info(t)))
    return d   